# Plug flow reactor simulation of Thruster

![caption](Graphics/thruster-details.png)


In [1]:
import cantera as ct
import numpy as np

from matplotlib import pyplot as plt
import csv
import os
import re
import itertools
import pandas as pd
from collections import defaultdict

In [2]:
# default if not using SLURM array
cat_area_per_vol = 3e6 # m2/m3
temperature_c = 400 # ºC
rtol = 1e-11
atol = 1e-24
residual_threshold = 5e-3

# input file containing the reaction mechanism
cti_file = '../RMG-model/cantera/chem_annotated.cti'
#cti_file = '../RMG-model/cantera/chem0050.cti'

from Collision_Violators

In [3]:
import os, sys
rmg_path = os.getenv('RMGpy')
if rmg_path not in sys.path:
    sys.path.append(rmg_path)
sys.path

import rmgpy
from rmgpy.chemkin import load_chemkin_file
from rmgpy.rmgobject import RMGObject, expand_to_dict, recursive_make_object

print(f"RMG-Py Version {rmgpy.__version__}")
print(rmgpy.__file__)

RDKit WARNING: [16:57:44] Enabling RDKit 2019.09.3 jupyter extensions


RMG-Py Version 3.0.0
/home/nadeau.ma/Code/RMG-Py/rmgpy/__init__.py


In [4]:
chemkin_file = '../RMG-model/chemkin/chem_annotated-surface.inp'
chemkin_folder = os.path.split(chemkin_file)[0]
species_dictionary_file = os.path.join(chemkin_folder, 'species_dictionary.txt')
transport_file = os.path.join(chemkin_folder, 'tran.dat')

In [5]:
species, reactions = rmgpy.chemkin.load_chemkin_file(
                chemkin_file, species_dictionary_file,transport_path=transport_file,
                check_duplicates=False, use_chemkin_names=True,read_comments=True)


compare
[Species(label="X(1)", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(0,'amu')), Species(label="HX(26)", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(1.00794,'amu')), Species(label="OX(27)", molecule=[Molecule(smiles="O=[Pt]")], molecular_weight=(15.9994,'amu')), Species(label="CH3X(28)", molecule=[Molecule(smiles="C[Pt]")], molecular_weight=(15.0345,'amu')), Species(label="HOX(29)", molecule=[Molecule(smiles="O[Pt]")], molecular_weight=(17.0073,'amu')), Species(label="H2OX(30)", molecule=[Molecule(smiles="O.[Pt]")], molecular_weight=(18.0153,'amu')), Species(label="CO2X(31)", molecule=[Molecule(smiles="O=C=O.[Pt]")], molecular_weight=(44.0095,'amu')), Species(label="OCX(32)", molecule=[Molecule(smiles="O=C=[Pt]")], molecular_weight=(28.0101,'amu')), Species(label="CX(33)", molecule=[Molecule(smiles="C~[Pt]")], molecular_weight=(12.0107,'amu')), Species(label="CH2X(34)", molecule=[Molecule(smiles="C=[Pt]")], molecular_weight=(14.0266,'amu')), Species(label="CHX(

reactants
C2H4(200)
Products
CH3X(28)
....
reactants
C2H4(200)
Products
CHX(35)
....
reactants
C2H4(200)
Products
SX(1484)
....
reactants
C2H4(200)
Products
HX(26)
....
reactants
X(1)
Products
SX(3063)
....
reactants
S(3047)
Products
SX(3063)
....
reactants
X(1)
Products
N2X(212)
....
reactants
N2(7)
Products
N2X(212)
....
reactants
X(1)
Products
COX(219)
....
reactants
CO(14)
Products
COX(219)
....
reactants
H2NO2(595)
Products
SX(933)
....
reactants
X(1)
Products
SX(933)
....
reactants
SX(1785)
Products
H2NX(201)
....
reactants
SX(1785)
Products
H2N2X(671)
....
reactants
H2NOX(203)
Products
SX(1785)
....
reactants
H2N2X(671)
Products
OX(27)
....
reactants
SX(1785)
Products
CH3NX(667)
....
reactants
CHX(35)
Products
H2N2X(671)
....
reactants
SX(1785)
Products
CH2NX(668)
....
reactants
CX(33)
Products
H2N2X(671)
....
reactants
SX(1785)
Products
CH4NX(666)
....
reactants
CH2X(34)
Products
H2N2X(671)
....
reactants
SX(1785)
Products
H2N2X(671)
....
reactants
NX(617)
Products
H2N2X(671)
.

In [6]:
#From chemkin:
print('there are {} reactions'.format(len(reactions)))
families_dict = {}
mass_transfer_limited = {}
keys = np.zeros(len(reactions))
for i in range(len(reactions)):
    rxn = reactions[i]
    keys[i] = rxn.index
#     print(keys[i],rxn,rxn.family)
    families_dict[i] = rxn.family
    if 'Adsorption' in rxn.family: 
        mass_transfer_limited.update({rxn.index : rxn.family})
    if 'Deutschmann2006' in rxn.family:
        mass_transfer_limited.update({rxn.index : rxn.family})
        

mass_transfer_limited.keys() #these have some from Deutschmann that are not mass transfer limited

there are 339 reactions


dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 31, 32, 33, 34, 35, 37, 38, 39, 56, 57, 58, 59, 60, 61, 68, 69, 70, 76, 80, 111, 112, 113, 128, 129, 130, 131, 148, 149, 150, 151, 152, 153, 154, 155, 166, 173, 174, 175, 176, 192, 210, 211, 212, 213, 215, 216, 221, 222, 228, 229, 230, 231, 232, 252, 256, 257, 281, 282, 283, 284, 285, 286, 287, 296, 336, 337, 338, 339])

In [7]:
duplicates = []
all_family_names = []
print('Surface reaction families:\n')
for i in families_dict:
    if families_dict[i] not in duplicates:
        print(families_dict[i])
        all_family_names.append(families_dict[i])
    duplicates.append(families_dict[i])

Surface reaction families:

Surface/CPOX_Pt/Deutschmann2006
Surface_Adsorption_vdW
Surface_Adsorption_Single
Surface_Adsorption_Dissociative
Surface_Dissociation
Surface_Abstraction


In [8]:
#cantera
gas=ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])

In [9]:
'''This way doesn't require chemkin or creating a dictionary in
   order to find which surface reactions involve gaseous species'''

rxn_numbers = []
gas_reactants = {}
test_if_morethan1 = []
print('These reactions use Arrhenius:')
for i in range(len(surf.reactions())):
    for (key,value) in surf.reaction(i).reactants.items():
                if key in gas.species_names: #if could be mass transfer limited
                    rxn_numbers.append(i)
                    gas_reactants.update({i:key})
                    test_if_morethan1.append(key)
#                     print(i+1,surf.reaction(i).is_sticking_coefficient)
                    if surf.reaction(i).is_sticking_coefficient == False:
                        print('    ',i+1,surf.reaction(i))
print('The rest use sticking coefficients.')
print('\nChecked that there only ever is 1 gaseous reactant in the adorption rxns: ',len(test_if_morethan1), len(gas_reactants.items()))
# print('\ni_Rxn  Gaseous Reactant')
# for(key,value) in gas_reactants.items():
#     print(' ',key+1,'   ',value)

These reactions use Arrhenius:
     11 O2(8) + 2 X(1) <=> 2 OX(27)
     150 CH4(15) + OX(27) + X(1) <=> CH3X(28) + HOX(29)
The rest use sticking coefficients.

Checked that there only ever is 1 gaseous reactant in the adorption rxns:  78 78


In [10]:
ads_kfs = [] 
for i in range(len(surf.forward_rate_constants)):
    if i in gas_reactants.keys(): 
        ads_kfs.append(surf.forward_rate_constants[i])
    
surf_adsorption_kfs=dict(zip(rxn_numbers, ads_kfs))

In [11]:
def stick_greater_than_1(T):
    '''
    For a given T  in Kelvin, returns a list of rxns for which Cantera 
    calculated the forward rate constant based on a sticking coefficient
    greater than 1
    '''
    violators = []
    sticks = []
    violator_rxn_indeces = []
    for j in range(len(surf.reactions())):

        # surf.reactant_stoich_coeffs() array of stoich coefficients. [k,i] is elements k in rxn i
        # surf.reactant_stoich_coeff(k,j) #stoich coeff of reactant k in rxn j
        for key in surf.reactions()[j].reactants:
            if key in gas.species_names:
                k = gas.species_names.index(key)
#                 print(surf.reaction(j))
#                 print(k)

                R = ct.gas_constant #J/kmol-K        
                m = sum(surf.reactant_stoich_coeffs()[:,j])-surf.reactant_stoich_coeff(k,j)
                gamma = surf.site_density #kmol/m^2
                W = gas.molecular_weights[k] #MW of gas kg/kmol
            #     print('m = {0}, MW = {1} #kg/kmol, species {2}'.format(m,W,gas.species_names[k]))
                stick = surf.forward_rate_constants[j]*gamma**m/(np.sqrt(R*T/(2*np.pi*gas.molecular_weights[k])))
                violation = stick > 1
            #     print(violation, stick)
#                 print(stick,'\n')
                if violation:
#                     print(surf.reaction(j))
#                     print(k)
#                     print('m = {0}, MW = {1} #kg/kmol, species {2}'.format(m,W,gas.species_names[k]))
#                     print('cantera kf is {:.1e}'.format(surf.forward_rate_constants[j]))
#             #     print("Calculated kf = cantera's kf?", test == surf.forward_rate_constants[j])
#                     print(violation, stick)
#                     print('\n')
                    violators.append(surf.reaction(j))
                    sticks.append(stick)
                    violator_rxn_indeces.append(j)

    #     print('kf is {:.1e}'.format(test))

    return violators,sticks,violator_rxn_indeces

# Temps = np.linspace(20+273.15,1000+273.15,num=5)
# for i in range(len(Temps)):
#     Temp = Temps[i]
#     rxns, values, indeces = stick_greater_than_1(Temp)
#     print(Temp-273.15, rxns, indeces,'\n')

# Temp = surf.T
# rxns, values, indeces = stick_greater_than_1(Temp)
# print("Reactions with sticking coeff > 1 :\n")
# print('Temp (C)             Rxns                 i_Rxn      Stick')
# for i in range(len(indeces)):
#     print('',Temp-273.15, '    ' ,rxns[i], '     ', np.array(indeces[i])+1, '     ','{:.3}'.format(values[i]))

In [12]:
print('Temp =', surf.T)
print("\nHighest forward rate constants, surface adsorption         k_f    stick?       Family")
n = 20
for k in sorted(surf_adsorption_kfs, key = surf_adsorption_kfs.get,reverse=True)[:n]:
    print(f"{k+1:3d} : {surf.reaction_equation(k):48s}  {surf_adsorption_kfs[k]:8.1e}  {str(surf.reaction(k).is_sticking_coefficient):5s}   {mass_transfer_limited[k+1]}")


Temp = 300.0

Highest forward rate constants, surface adsorption         k_f    stick?       Family
  6 : H2(13) + 2 X(1) <=> 2 HX(26)                       3.3e+16  True    Surface/CPOX_Pt/Deutschmann2006
 11 : O2(8) + 2 X(1) <=> 2 OX(27)                        1.1e+16  False   Surface/CPOX_Pt/Deutschmann2006
151 : CH4(15) + HOX(29) + X(1) <=> CH3X(28) + H2OX(30)   4.6e+15  True    Surface/CPOX_Pt/Deutschmann2006
  1 : H2O(6) + X(1) <=> H2OX(30)                         4.5e+09  True    Surface/CPOX_Pt/Deutschmann2006
173 : CO(14) + X(1) <=> OCX(32)                          4.0e+09  True    Surface/CPOX_Pt/Deutschmann2006
  7 : H(20) + X(1) <=> HX(26)                            2.5e+09  True    Surface_Adsorption_Single
 33 : H2(13) + X(1) <=> H2X(218)                         1.8e+09  True    Surface_Adsorption_vdW
150 : CH4(15) + OX(27) + X(1) <=> CH3X(28) + HOX(29)     1.3e+09  False   Surface/CPOX_Pt/Deutschmann2006
222 : NO3(95) + X(1) <=> NO3X(207)                       9.7e+08  T

In [13]:
#Figuring Out How Cantera calculates kf from sticking coefficient 
print(gas_reactants[5],surf_adsorption_kfs[5])
print('Cantera kf:',surf.forward_rate_constants[5], 'rxn:',surf.reaction_equation(5))
print('gas is {0} with molecular weight {1} kg/kmol'.format(gas.species(12), gas.molecular_weights[12]))
gamma = surf.site_density #kmol/m^2
R = ct.gas_constant #J/kmol-K
surf.T #K
np.pi
gas.species(12), gas.molecular_weights[12] #kg/kmol
m = 2 #sum stoich coeffs of 'surface reactants' aka empty sites 
test = 4.600000e-02/gamma**2*np.sqrt(R*surf.T/(2*np.pi*gas.molecular_weights[12]))

print('Calculated kf is {:.1e}'.format(test))
print("Calculated kf = cantera's kf?", test == surf.forward_rate_constants[5])

test - surf.forward_rate_constants[5]

H2(13) 3.3110049244775816e+16
Cantera kf: 3.3110049244775816e+16 rxn: H2(13) + 2 X(1) <=> 2 HX(26)
gas is <Species H2(13)> with molecular weight 2.01588 kg/kmol
Calculated kf is 3.3e+16
Calculated kf = cantera's kf? False


-4.0

In [14]:
# surf.reactant_stoich_coeffs() array of stoich coefficients. [k,i] is elements k in rxn i
# surf.reactant_stoich_coeff(k,j) #stoich coeff of reactant k in rxn j
j = 5

for key in surf.reactions()[j].reactants:
    if key in gas.species_names:
        k = gas.species_names.index(key)
        print(surf.reaction(j))
        print(k)

R = ct.gas_constant #J/kmol-K        
m = sum(surf.reactant_stoich_coeffs()[:,j])-surf.reactant_stoich_coeff(k,j)
gamma = surf.site_density #kmol/m^2
W = gas.molecular_weights[k] #MW of gas kg/kmol
print('m = {0}, MW = {1} #kg/kmol, species {2}'.format(m,W,k+1))
test = 4.600000e-02/gamma**m*np.sqrt(R*surf.T/(2*np.pi*gas.molecular_weights[k]))

print('Calculated kf is {:.1e}'.format(test))
print("Calculated kf = cantera's kf?", test == surf.forward_rate_constants[j])

H2(13) + 2 X(1) <=> 2 HX(26)
12
m = 2.0, MW = 2.01588 #kg/kmol, species 13
Calculated kf is 3.3e+16
Calculated kf = cantera's kf? False


In [15]:
print(np.shape(surf.reactant_stoich_coeffs()))
len(surf.reactions()),len(gas.species())

(246, 339)


(339, 177)

In [16]:
print(f"Catalyst area per volume {cat_area_per_vol :.2e} m2/m3")
print(f"Initial temperature      {temperature_c :.1f} ºC")
print(f"Solver RTOL              {rtol :.1e}")
print(f"Solvel ATOL              {atol :.1e}")

Catalyst area per volume 3.00e+06 m2/m3
Initial temperature      400.0 ºC
Solver RTOL              1.0e-11
Solvel ATOL              1.0e-24


In [17]:
gas()


  gas:

       temperature             300  K
          pressure          101325  Pa
           density         0.81974  kg/m^3
  mean mol. weight         20.1797  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          1905.6        3.845e+04     J
   internal energy      -1.217e+05       -2.456e+06     J
           entropy          7257.7        1.465e+05     J/K
    Gibbs function     -2.1754e+06        -4.39e+07     J
 heat capacity c_p          1030.1        2.079e+04     J/K
 heat capacity c_v          618.03        1.247e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                Ne              1                1         -17.5994
     [ +176 minor]              0                0



In [18]:
print(", ".join(gas.species_names))

Ne, NH3(2), NH2OH(3), HNO3(4), CH3OH(5), H2O(6), N2(7), O2(8), NO2(9), NO(10), N2O(11), CO2(12), H2(13), CO(14), CH4(15), C2H6(16), CH2O(17), CH3(18), C3H8(19), H(20), C2H5(21), HCO(22), CH3CHO(23), OH(24), C2H4(25), O(36), Ar(37), HO2(39), H2O2(40), HOCO(41), CH2(42), CH2(S)(43), CH(44), CH2OH(45), CH3O(46), HCOH(47), CH3OO(48), CH2CO(49), C2H3(50), C(51), C2H2(52), C2H(53), CH3OOH(54), CH2OOH(55), HOCH2O(56), HOCHO(57), C2H5O(58), C2H5O2(59), C2H5O2(60), cC2H4O(61), CH2CHO(62), H2CC(63), CH3CO(64), C2H4O(65), C2H5O(66), C2H3O2(67), CHCHO(68), OCHCHO(69), HCCO(70), HCCOH(71), CHCHOH(72), C2(73), C2O(74), C2H6O(75), C2H5O(76), C2H5O3(77), cC2H3O(78), C2H3O3(79), OCHCO(80), C2H6O2(81), C2H5O2(82), C2H4O2(83), OCHO(84), NH2(85), NH(86), HNO(87), H2NO(88), HON(89), N(90), NNH(91), HONO(92), HNOH(93), HNO2(94), NO3(95), N2H2(96), H2N2(97), N2H3(98), N2H4(99), HCN(100), CN(101), HNC(102), NCO(103), HOCN(104), HNCO(105), NCCN(106), HNCN(107), NCN(108), HNCNH(109), HCNO(110), CH3CN(111), CH2C

In [19]:
print(", ".join(surf.species_names))

X(1), HX(26), OX(27), CH3X(28), HOX(29), H2OX(30), CO2X(31), OCX(32), CX(33), CH2X(34), CHX(35), H2NX(201), H3NX(202), H2NOX(203), H2NOX(204), H3NOX(205), NO2X(206), NO3X(207), HNO3X(208), CH3OX(209), CH3OX(210), CH4OX(211), N2X(212), NO2X(214), NOX(215), NOJX(216), H2X(218), COX(219), CH4X(220), C2H5X(221), CH2OX(224), HNX(519), HNOX(520), CH2OX(522), H2NOX(532), H2NOX(533), CHOX(588), SX(589), SX(590), HNO2X(594), NX(617), SX(618), CH4NX(666), CH3NX(667), CH2NX(668), SX(670), H2N2X(671), HONOX(672), HNOX(734), HNOX(744), H3N2X(930), SX(933), SX(935), CNOX(943), N2OX(946), SX(955), SX(1141), SX(1252), SX(1483), SX(1484), SX(1785), SX(2178), SX(2200), SX(2238), SX(2261), SX(2264), SX(2324), SX(2374), SX(3063)




This example solves a plug flow reactor problem, with coupled surface and gas chemistry.





In [20]:
# unit conversion factors to SI
cm = 0.01 # m
minute = 60.0  # s

In [21]:
#######################################################################
# Input Parameters for combustor
#######################################################################
mass_flow_rate =  0.5e-3 # kg/s
#temperature_c = 550.0  # Initial Temperature in Celsius
print(f"Initial temperature {temperature_c :.1f} ºC")
pressure = ct.one_atm # constant

length = 1.1 * cm  # Catalyst bed length. 11mm
cross_section_area = np.pi * (0.9*cm)**2  # Catalyst bed area.  18mm diameter circle.

### Catalyst properties. Some are hard to estimate
# if we can, update this lit value or verify the value richard calculated
porosity = 0.38  # Catalyst bed porosity (0.38)
# Al2O3 particles are about 0.7mm diameter
cat_specific_area = 140 # m2/g
print(f"Catalyst specific area {cat_specific_area :.2e} m2/g")
cat_density = 2 / cm**3 # 2 g/m3
print(f"Catalyst density {cat_density :.2e} g/m3")
cat_area_per_reactor_vol = cat_specific_area * cat_density # m2/m3
print(f"Catalyst area per total reactor volume {cat_area_per_reactor_vol :.2e} m/m3")
cat_area_per_gas_vol = cat_area_per_reactor_vol / porosity # porosity is gas vol per reactor vol
print(f"Catalyst area per gas volume {cat_area_per_gas_vol :.2e} m/m3")

#cat_area_per_vol =  cat_area_per_gas_vol * 1e-3 # REDUCE BY A LOT
print(f"\nCatalyst area per volume in use for this simulation: {cat_area_per_vol :.2e} m2/m3")



Initial temperature 400.0 ºC
Catalyst specific area 1.40e+02 m2/g
Catalyst density 2.00e+06 g/m3
Catalyst area per total reactor volume 2.80e+08 m/m3
Catalyst area per gas volume 7.37e+08 m/m3

Catalyst area per volume in use for this simulation: 3.00e+06 m2/m3


In [22]:
# output_filename = 'surf_pfr_output_mlou_test.csv'

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 2201

#####################################################################

temperature_kelvin = temperature_c + 273.15  # convert to Kelvin

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')

# From HAN-molefractions.ipynb
feed_mole_fractions = {
    'NH3(2)': 0.031,
    'NH2OH(3)': 0.32,
    'HNO3(4)': 0.35,
    'CH3OH(5)': 0.21,
    'H2O(6)': 0.09,
}
gas.TPX = temperature_kelvin, pressure, feed_mole_fractions


# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = temperature_kelvin, pressure
surf.coverages = 'X(1):1.0'

r_len = length/(NReactors-1) 
r_vol = cross_section_area * r_len * porosity # gas volume

# catalyst area in one reactor
cat_area = cat_area_per_vol * r_vol

# Not sure we need the velocity
velocity = mass_flow_rate / (gas.density * cross_section_area)

In [23]:
def report_rates(n=8):
    print("\nHighest net rates of progress, gas")
    for i in np.argsort(abs(gas.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {gas.reaction_equation(i):48s}  {gas.net_rates_of_progress[i]:8.1g}")
    print("\nHighest net rates of progress, surface")
    for i in np.argsort(abs(surf.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.net_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, gas")
    for i in np.argsort(abs(gas.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, surface")
    for i in np.argsort(abs(surf.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, gas")
    for i in np.argsort(abs(gas.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, surface")
    for i in np.argsort(abs(surf.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.reverse_rates_of_progress[i]:8.1g}")

    print(f"\nSurface rates have been scaled by surface/volume ratio {cat_area_per_vol:.1e} m2/m3")
    print("So are on a similar basis of volume of gas")
    print(" kmol / m3 / s")
# report_rates()

In [24]:
def report_rate_constants(n=8):
    print("\nHighest forward rate constants, gas")
    for i in np.argsort(abs(gas.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rate_constants[i]:8.1e}")
    print("\nHighest forward rate constants, surface")
    for i in np.argsort(abs(surf.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {surf.reaction_equation(i):48s}  {surf.forward_rate_constants[i]:8.1e}")
        print(surf.reaction(i).is_sticking_coefficient)
    print("\nHighest reverse rate constants, gas")
    for i in np.argsort(abs(gas.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, surface")
    for i in np.argsort(abs(surf.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i+1:3d} : {surf.reaction_equation(i):48s}  {surf.reverse_rate_constants[i]:8.1e}")

    print("Units are a combination of kmol, m^3 and s, that depend on the rate expression for the reaction.")
# report_rate_constants()

In [25]:
def fix_rates(phase, limit):
    """
    Fix reverse reaction rates that are too fast.
    """
    for i in np.argsort(abs(phase.reverse_rate_constants))[-1:0:-1]:
        if phase.reverse_rate_constants[i] < limit:
            break
#         print(f"Before: {i:3d} : {phase.reaction_equation(i):48s}  {phase.reverse_rate_constants[i]:8.1e}")
        multiplier = limit / phase.reverse_rate_constants[i]
        phase.set_multiplier(multiplier, i)
#         print(f"After:  {i:3d} : {phase.reaction_equation(i):48s}  {phase.reverse_rate_constants[i]:8.1e}")
        
#fix_rates(gas, 1e18)
#fix_rates(surf, 1e21)

In [26]:
def save_flux_diagrams(*phases, suffix=''):
    """
    Saves the flux diagrams. The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    """
    for element in 'CHONX':
        for phase_object in phases:
            phase = phase_object.name

            diagram = ct.ReactionPathDiagram(phase_object, element)
            diagram.title = f'Reaction path diagram following {element} in {phase}'
            diagram.label_threshold = 0.01

            dot_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.dot"
            img_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            img_path = os.path.join(os.getcwd(), img_file)
            diagram.write_dot(dot_file)
            #print(diagram.get_data())

            print(f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'.")
            os.system(f'dot {dot_file} -Tpng -o{img_file} -Gdpi=200')
            print(f"Wrote graphviz output file to '{img_path}'.")

def show_flux_diagrams(*phases, suffix='', embed=False):
    """
    Shows the flux diagrams in the notebook.
    Loads them from disk.
    Does not embed them, to keep the .ipynb file small,
    unless embed=True. Use embed=True if you might over-write the files,
    eg. you want to show flux at different points.
    """
    import IPython
    for element in 'CHONX':
        for phase_object in phases:
            phase = phase_object.name
            img_file = f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            display(IPython.display.HTML(f'<hr><h2>{element} {phase}</h2>'))
            if embed:
                display(IPython.display.Image(filename=img_file,width=400,embed=True))
            else:
                display(IPython.display.Image(url=img_file,width=400,embed=False))


def integrated_flux_diagrams():
    """This is a code fragment. Not working. Do not use it."""
    for element in 'CHON':
        diagrams = [ct.ReactionPathDiagram(surf, element), ct.ReactionPathDiagram(gas, element)]
        for diagram in diagrams:
            data = diagram.get_data()
            split_data = data.split("\n")
            for line in split_data[2:]:
                if len(line.split()) == 0: # skip empty line
                    continue
                s1, s2, fwd, rev = line.split()
                net = float(fwd) - float(rev)
                if net == 0.0:
                    continue
                flux_pair = (s1, s2)
                integration_flux_data[flux_pair] += net
    

In [27]:
gas.TPX = temperature_kelvin, pressure, feed_mole_fractions
surf.coverages = 'X(1):1.0'
#surf.coverages = starting_coverages

In [28]:
# The plug flow reactor is represented by a linear chain of zero-dimensional
# reactors. The gas at the inlet to the first one has the specified inlet
# composition, and for all others the inlet composition is fixed at the
# composition of the reactor immediately upstream. Since in a PFR model there
# is no diffusion, the upstream reactors are not affected by any downstream
# reactors, and therefore the problem may be solved by simply marching from
# the first to last reactor, integrating each one to steady state.

TDY = gas.TDY
cov = surf.coverages

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas, energy='on')
r.volume = r_vol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 24

# set relative and absolute tolerances on the simulation
sim.rtol = rtol
sim.atol = atol

sim.verbose = False

# surf.set_multiplier(0.)  # turn off surface reactions
# surf.set_multiplier(1e6)  # make surface reactions a million times faster

r.volume = r_vol
rsurf.area = cat_area

integration_flux_data = defaultdict(float)

# outfile = open(output_filename,'w')
# writer = csv.writer(outfile)
# writer.writerow(['Distance (mm)', 'T (C)', 'P (atm)'] +
#                 gas.species_names + surf.species_names + ['gas_heat','surface_heat','alpha'])

print('   distance(mm)  r.T (C)    gas.T (C)    NH3(2)   NH2OH(3)     HNO3(4)    CH3OH(5)  alpha')

violating_rxns = []
violating_sticks = []
violating_indeces = []

for n in range(NReactors):
    """
    if n == 0: # first coulpe of reactors are tiny
        surf.set_multiplier(0.)
        r.volume = r_vol * 1e-2
        rsurf.area = cat_area * 1e-2
    if n == 3:
        r.volume = r_vol
        rsurf.area = cat_area"""
        
    if n == 0: # start off with inert packing, no surface reactions
        surf.set_multiplier(0.)
    if n == int(0.001 * NReactors / length): # after 1 mm, catalyst
        surf.set_multiplier(1.)
        
    #testing set_multiplier for specific reactions:  
#     surf.set_multiplier(.0001,i_reaction=0) #this sets rxn 0's k to .0001%
#     can't pass this a list
#     for key,value in surf_adsorption_kfs:
#         if i in gas_reactants.keys():
#             gas.set_multiplier(multipliers[i],i_reaction=i)
#         print(gas.forward_rate_constants[i],families_dict[i])  
    
    fix_rates(surf, 1e22)
#         save_flux_diagrams(gas, surf, suffix='1mm')
#         show_flux_diagrams(gas, surf, suffix='1mm', embed=True)
    
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY
    cov = surf.coverages
    upstream.syncState()
    sim.reinitialize()
    try:
#       the default is residual_threshold = sim.rtol*10
        sim.advance_to_steady_state(residual_threshold = residual_threshold)

    except ct.CanteraError:
        t = sim.time
        sim.set_initial_time(0)
        gas.TDY = TDY
        surf.coverages = cov
        r.syncState()
        sim.reinitialize()
        new_target_time = 0.01 * t
        print(f"Couldn't reach {t:.1g} s so going to try {new_target_time:.1g} s")
        #save_flux_diagrams(gas, surf)
        #show_flux_diagrams(gas, surf, embed=True)
        report_rates()
        report_rate_constants()
        try:
            sim.advance(new_target_time)
        except ct.CanteraError:
            outfile.close()
            raise()
            #report_rate_constants()
 
    dist = n * r_len * 1.0e3   # distance in mm
        
    gas_heat = np.dot(gas.net_rates_of_progress, gas.delta_enthalpy) # heat evolved by gas phase reaction
    surface_heat = cat_area_per_vol * np.dot(surf.net_rates_of_progress, surf.delta_enthalpy) # heat evolved by surf phase reaction 
    alpha = surface_heat / (surface_heat + gas_heat) # fraction of heat release that is on surface.

    if not n % 50:
        print('    {:10f}  {:7.1f}  {:7.1f}  {:10f}  {:10f}  {:10f} {:10f}  {:5.1e}'.format(dist, r.T-273.15,gas.T-273.15, *gas['NH3(2)','NH2OH(3)','HNO3(4)','CH3OH(5)'].X, alpha ))
#         print('{:7.1f}  {:7.1f}  {:7.1f}'.format(r.T-273.15,gas.T-273.15,surf.T-273.15))
    # write the gas mole fractions and surface coverages vs. distance
#         writer.writerow([dist, r.T - 273.15, r.thermo.P/ct.one_atm] +
#                     list(gas.X) + list(surf.coverages) + [gas_heat, surface_heat, alpha])

    rxns, values, indeces= stick_greater_than_1(surf.T)
    
    violating_rxns.append(rxns)
    violating_sticks.append(values)
    violating_indeces.append(indeces)
    

# outfile.close()
# print("Results saved to '{0}'".format(output_filename))

# with open("integration_flux_data.txt",'w') as f:
#     for (sp1,sp2),flux in integration_flux_data.items():
#         f.write("{} {} {}\n".format(sp1,sp2,flux))
            

   distance(mm)  r.T (C)    gas.T (C)    NH3(2)   NH2OH(3)     HNO3(4)    CH3OH(5)  alpha
      0.000000    400.0    400.0    0.030969    0.319680    0.349650   0.209790  0.0e+00
      0.250000    400.0    400.0    0.030969    0.319680    0.349650   0.209790  0.0e+00
      0.500000    400.0    400.0    0.030969    0.319680    0.349650   0.209790  0.0e+00


KeyboardInterrupt: 

In [31]:
count = 0
for i in range(len(violating_rxns)):
    if violating_rxns[i] != []:
        print(violating_rxns[i])
    else:
        count+=1
count

116

In [32]:
#Initial testing for sticking coefficients at T of rxn
violators = []
for j in range(len(surf.reactions())):

    for key in surf.reactions()[j].reactants:
        if key in gas.species_names:
            k = gas.species_names.index(key)
            print(surf.reaction(j))
            print('species index:',k,'\nchemkin rxn num:',j+1)
            T = surf.T
            R = ct.gas_constant #J/kmol-K   
            # surf.reactant_stoich_coeffs() array of stoich coefficients. [k,i] is elements k in rxn i
            # surf.reactant_stoich_coeff(k,j) #stoich coeff of reactant k in rxn j
            #m is the sum of the species on the surface's stoichiometric coefficients
            m = sum(surf.reactant_stoich_coeffs()[:,j])-surf.reactant_stoich_coeff(k,j)
            gamma = surf.site_density #kmol/m^2
            W = gas.molecular_weights[k] #MW of gas kg/kmol
            print('m = {0}, MW = {1} #kg/kmol, species {2}'.format(m,W,gas.species_names[k]))
            stick = surf.forward_rate_constants[j]*gamma**m/(np.sqrt(R*T/(2*np.pi*gas.molecular_weights[k])))
            violation = stick > 1
            print('Violator?',violation,'. Stick:{:.2}'.format(stick), '\n')
            if violation:
#                 print(surf.reaction(j))
#                 print(k)
#                 print('m = {0}, MW = {1} #kg/kmol, species {2}'.format(m,W,gas.species_names[k]))
#                 print('cantera kf is {:.1e}'.format(surf.forward_rate_constants[j]))
#         #     print("Calculated kf = cantera's kf?", test == surf.forward_rate_constants[j])
#                 print(violation, stick)
#                 print('\n')
                violators.append(surf.reaction(j))
    
violators

H2O(6) + X(1) <=> H2OX(30)
species index: 5 
chemkin rxn num: 1
m = 1.0, MW = 18.01528 #kg/kmol, species H2O(6)
Violator? False . Stick:0.0 

NH2OH(3) + X(1) <=> H3NOX(205)
species index: 2 
chemkin rxn num: 2
m = 1.0, MW = 33.02996 #kg/kmol, species NH2OH(3)
Violator? False . Stick:0.0 

HNO3(4) + X(1) <=> HNO3X(208)
species index: 3 
chemkin rxn num: 3
m = 1.0, MW = 63.012879999999996 #kg/kmol, species HNO3(4)
Violator? False . Stick:0.0 

CH3OH(5) + X(1) <=> CH4OX(211)
species index: 4 
chemkin rxn num: 4
m = 1.0, MW = 32.04216 #kg/kmol, species CH3OH(5)
Violator? False . Stick:0.0 

NH3(2) + X(1) <=> H3NX(202)
species index: 1 
chemkin rxn num: 5
m = 1.0, MW = 17.03056 #kg/kmol, species NH3(2)
Violator? False . Stick:0.0 

H2(13) + 2 X(1) <=> 2 HX(26)
species index: 12 
chemkin rxn num: 6
m = 2.0, MW = 2.01588 #kg/kmol, species H2(13)
Violator? False . Stick:0.0 

H(20) + X(1) <=> HX(26)
species index: 19 
chemkin rxn num: 7
m = 1.0, MW = 1.00794 #kg/kmol, species H(20)
Violator? F

species index: 20 
chemkin rxn num: 232
m = 1.0, MW = 29.0617 #kg/kmol, species C2H5(21)
Violator? False . Stick:0.0 

S(1095) + 2 X(1) <=> HOX(29) + SX(1141)
species index: 169 
chemkin rxn num: 252
m = 2.0, MW = 77.03976 #kg/kmol, species S(1095)
Violator? False . Stick:0.0 

H3NO3(627) + 2 X(1) <=> HOX(29) + SX(933)
species index: 165 
chemkin rxn num: 256
m = 2.0, MW = 65.02876 #kg/kmol, species H3NO3(627)
Violator? False . Stick:0.0 

S(3047) + 2 X(1) <=> NO2X(206) + SX(933)
species index: 174 
chemkin rxn num: 257
m = 2.0, MW = 94.02696 #kg/kmol, species S(3047)
Violator? False . Stick:0.0 

S(3054) + 2 X(1) <=> 2 SX(933)
species index: 175 
chemkin rxn num: 281
m = 2.0, MW = 96.04284 #kg/kmol, species S(3054)
Violator? False . Stick:0.0 

C2H4(200) + 2 X(1) <=> CH3X(28) + CHX(35)
species index: 149 
chemkin rxn num: 282
m = 2.0, MW = 28.053759999999997 #kg/kmol, species C2H4(200)
Violator? False . Stick:0.0 

C2H4(200) + 2 X(1) <=> HX(26) + SX(1484)
species index: 149 
chemkin r

[]

In [ ]:
gas.TDY = TDY
r.syncState()
r.thermo.T

In [ ]:
data = pd.read_csv(output_filename)
data

In [ ]:
def xlabels():
    ticks = []
    labels = []
    mm = 0
    while mm < length*1000:
        ticks.append( int(NReactors * mm * 0.001 / length ) )
        labels.append( str(mm) )
        mm += 1
    labels[-1] = labels[-1] + ' mm'
    plt.xticks(ticks, labels)
    plt.xlabel("Distance down reactor")

In [ ]:
data['T (C)'].plot()
plt.ylabel('T (C)')
xlabels()

In [ ]:
data[['NH2OH(3)', 'HNO3(4)', 'CH3OH(5)']].plot()
plt.ylabel('Mole fraction')
xlabels()

In [ ]:
list(data.columns)[:4]

In [ ]:
data[['T (C)', 'alpha']].plot()
xlabels()

In [ ]:
ax1 = data['T (C)'].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.legend()
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, color='tab:orange')
ax2.set_ylim(-2, 2)
plt.legend()
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('temperature-and-alpha.pdf')
plt.show()

In [ ]:
data.columns

In [ ]:
data[['gas_heat','surface_heat']].plot()
#plt.ylim(-1e7, 1e7)
xlabels()
plt.savefig('gas_and_surface_heat.pdf')
plt.show()


In [ ]:
ax1 = data[['gas_heat','surface_heat']].plot()
plt.ylim(-1e9, 1e9)
xlabels()
plt.ylabel('Heat consumption rate (kJ/m3/s)')
plt.legend(loc='upper left')
ax2 = ax1.twinx()
data['alpha'].plot(ax=ax2, style='k:', alpha=0.5)
ax2.set_ylim(-10, 10)
plt.legend(loc='lower right')
plt.ylabel('alpha')
plt.tight_layout()
plt.savefig('heats-and-alpha.pdf')
plt.show()

In [ ]:
data[['T (C)']].plot()
plt.ylabel('Temperature (C)')
xlabels()
plt.tight_layout()
plt.savefig('temperature.pdf')
plt.show()

In [ ]:
data[['alpha']].plot(logy=True)
xlabels()

In [ ]:
data.plot(x='T (C)',y='alpha')


In [ ]:
specs = list(data.columns)
specs = specs[4:-3]
excluded = [s for s in data.columns if s not in specs]
gas_species = [s for s in specs if 'X' not in s ]
adsorbates = [s for s in specs if 'X' in s]

excluded, gas_species, adsorbates

In [ ]:
data[gas_species[0:5]].plot(logy=True, logx=True)

In [ ]:
for i in range(0,len(gas_species),10):
    data[gas_species[i:i+10]].plot(title='Gas mole fraction', logy=False)
    xlabels()
    plt.ylabel('Mole fraction')
    plt.tight_layout()
    plt.savefig(f'gas_mole_fractions_{i}.pdf')
    plt.show()
    


In [ ]:
for i in range(0,len(adsorbates),10):
    data[adsorbates[i:i+10]].plot(title='Surface coverages', logy=False)
    xlabels()
    plt.xlim(0,len(data)+5)
    plt.ylabel('Surface coverage')
    plt.tight_layout()
    plt.savefig(f'surface_coverages_{i}.pdf')
    plt.show()

In [ ]:
main_gas_species = data[gas_species].max().sort_values(ascending=False)[:10].keys()
data[main_gas_species].plot.area()

xlabels()
plt.xlim(0,len(data)+5)
plt.tight_layout()
plt.savefig(f'gas_mole_fractions_top10.pdf')
plt.show()

In [ ]:
main_adsorbates = data[adsorbates].max().sort_values(ascending=False)[:10].keys()
data[main_adsorbates].plot.area()

xlabels()
plt.xlim(190,len(data)+5)
plt.tight_layout()
plt.savefig(f'surface_coverages_top10.pdf')
plt.show()
    

In [ ]:
for a in main_adsorbates:
    s = surf.species(a)
    print(s, s.composition)

In [ ]:
surf.coverages

In [ ]:
surf.set_multiplier(1)
diagram = ct.ReactionPathDiagram(surf, 'X')
diagram.get_data()